# Hybrid Recommendation using LightFM

<img src="https://analyticsindiamag.com/wp-content/uploads/2021/10/image-97.png" width="800" height="400">

# Implementation

# Importing the required libraries

In [1]:
!pip install lightfm

  Using cached lightfm-1.16.tar.gz (310 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
     -------------------------------------- 160.2/160.2 kB 1.6 MB/s eta 0:00:00
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Running setup.py clean for lightfm
Failed to build lightfm
  Running setup.py install for lightfm: started
  Running setup.py install for lightfm: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [25 lines of output]
  Compiling without OpenMP support.
  C:\Users\vadi_\anaconda3\envs\suprisepackage\lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
    warnings.warn(
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\lightfm
  copying lightfm\cross_validation.py -> build\lib.win-amd64-cpython-310\lightfm
  copying lightfm\data.py -> build\lib.win-amd64-cpython-310\lightfm
  copying lightfm\evaluation.py -> build\lib.win-amd64-cpython-310\lightfm
  copying lightfm\lightfm.py -> build\lib.win-amd64-cpython-310\lightfm
  copying lightfm\_lightfm_fast.py -> build\lib.win-amd64-cpython-310\lightfm
  copying lightfm\_

In [3]:
import pandas as pd # pandas for data manipulation
import numpy as np # numpy for sure
from scipy.sparse import coo_matrix # for constructing sparse matrix
# lightfm 
from lightfm import LightFM # model
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split

# timing
import time

ModuleNotFoundError: No module named 'lightfm'

# Importing the data from excel

In [4]:
order_df = pd.read_excel('Rec_sys_data.xlsx','order')
customer_df = pd.read_excel('Rec_sys_data.xlsx','customer')
product_df = pd.read_excel('Rec_sys_data.xlsx','product')

# Merging the datasets

In [10]:
merged_df = pd.merge(order_df,customer_df,left_on=['CustomerID'], right_on=['CustomerID'], how='left')
merged_df = pd.merge(merged_df,product_df,left_on=['StockCode'], right_on=['StockCode'], how='left')

In [6]:
order_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850


In [7]:
customer_df.head()

,CustomerID,Gender,Age,Income,Zipcode,Customer Segment
0,13089,male,53,High,8625,Small Business
1,15810,female,22,Low,87797,Small Business
2,15556,female,29,High,29257,Corporate
3,13137,male,29,Medium,97818,Middle class
4,16241,male,36,Low,79200,Small Business


In [8]:
product_df.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [11]:
merged_df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,DeliveryDate,Discount%,ShipMode,ShippingCost,CustomerID,Gender,Age,Income,Zipcode,Customer Segment,Product Name,Description,Category,Brand,Unit Price
0,536365,84029E,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.20,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,"3 1/2""W x 20""D x 20""H Funston Craftsman Smooth...",Our Rustic Collection is an instant classic. O...,Home Improvement|Hardware|Brackets and Angle I...,Ekena Milwork,199.11
1,536365,71053,6,2010-12-01 08:26:00,2010-12-02 08:26:00,0.21,ExpressAir,30.12,17850,female,48,Medium,84306,Middle class,Awkward Styles Shamrock Flag St. Patrick's Day...,Our St Patrick's Day Collection is perfect for...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens Gr...,Awkward Styles,23.95
2,536365,21730,6,2010-12-01 08:26:00,2010-12-03 08:26:00,0.56,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,Ebe Men Black Rectangle Half Rim Spring Hinge ...,Count on EBE for all of your eye correction ne...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,26.99
3,536365,84406B,8,2010-12-01 08:26:00,2010-12-03 08:26:00,0.30,Regular Air,15.22,17850,female,48,Medium,84306,Middle class,MightySkins Skin Decal Wrap Compatible with Ap...,Mightyskins are removable vinyl skins for prot...,Electronics|Electronics Learning Center|Ads Fr...,Mightyskins,14.99
4,536365,22752,2,2010-12-01 08:26:00,2010-12-04 08:26:00,0.57,Delivery Truck,5.81,17850,female,48,Medium,84306,Middle class,awesome since 1948 - 69th birthday gift t-shir...,awesome since 1948 - 69th birthday gift t-shir...,Clothing|Men|Mens T-Shirts & Tank Tops|Mens T-...,Shirtinvaders,49.33


In [60]:
# Creating the list of unique users
def unique_users(data, column):   
    return np.sort(data[column].unique())

# Creating the list of unique produts
def unique_items(data, column):
    item_list = data[column].unique()
    return item_list

def features_to_add(customer, column1,column2,column3):
    customer1 = customer[column1]
    customer2 = customer[column2]
    customer3 = customer[column3]
    return pd.concat([customer1,customer3,customer2], ignore_index = True).unique()

# Create id mappings to convert user_id, item_id, and feature_id
def mapping(user_list, item_list, feature_unique_list):
    #creating empty output dicts
    user_to_index_mapping = {}
    index_to_user_mapping = {}
    # Create id mappings to convert user_id
    for user_index, user_id in enumerate(user_list):
        user_to_index_mapping[user_id] = user_index
        index_to_user_mapping[user_index] = user_id
        
    item_to_index_mapping = {}
    index_to_item_mapping = {}
    # Create id mappings to convert item_id
    for item_index, item_id in enumerate(item_list):
        item_to_index_mapping[item_id] = item_index
        index_to_item_mapping[item_index] = item_id
        
    feature_to_index_mapping = {}
    index_to_feature_mapping = {}
    # Create id mappings to convert feature_id
    for feature_index, feature_id in enumerate(feature_unique_list):
        feature_to_index_mapping[feature_id] = feature_index
        index_to_feature_mapping[feature_index] = feature_id
        
    return user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping



In [42]:
# create the user, item, feature lists
user_list = unique_users(order_df, "CustomerID")
item_list = unique_items(product_df, "Product Name")
feature_unique_list = features_to_add(customer_df,'Customer Segment',"Age","Gender")

In [16]:
user_list

array([12346, 12347, 12348, ..., 18282, 18283, 18287], dtype=int64)

In [17]:
item_list

array(['Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover',
       'Eye Buy Express Prescription Glasses Mens Womens Burgundy Crystal Clear Yellow Rounded Rectangular Reading Glasses Anti Glare grade',
       "MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options",
       'Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches',
       'Stupell Industries Chevron Initial Wall D cor',
       "MightySkins Skin Decal Wrap Compatible with OtterBox Sticker Protective Cover 100's of Color Options",
       "MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options",
       'MightySkins Skin For OtterBox Commuter iPhone 8 Plus | Protective, Durable, and Unique Vinyl Decal wrap cover | Easy To Apply, Remove, and Change Styles | Made in the USA',
       "Dr. Comfort Paradise Women's Casual Shoe: 4.5 X-Wide (E-2E) Black Velcro",
       "Mig

In [43]:
feature_unique_list

array(['Small Business', 'Corporate', 'Middle class', 'male', 'female',
       53, 22, 29, 36, 48, 45, 47, 23, 39, 34, 52, 51, 35, 19, 26, 37, 18,
       20, 21, 41, 31, 28, 50, 38, 30, 25, 32, 55, 43, 54, 49, 40, 33, 44,
       46, 42, 27, 24], dtype=object)

In [44]:
# generate mapping, LightFM library can't read other than (integer) index
user_to_index_mapping, index_to_user_mapping, \
           item_to_index_mapping, index_to_item_mapping, \
           feature_to_index_mapping, index_to_feature_mapping = mapping(user_list, item_list, feature_unique_list)

In [45]:
user_to_index_mapping

{12346: 0,
 12347: 1,
 12348: 2,
 12350: 3,
 12352: 4,
 12353: 5,
 12354: 6,
 12355: 7,
 12356: 8,
 12358: 9,
 12359: 10,
 12360: 11,
 12361: 12,
 12362: 13,
 12363: 14,
 12364: 15,
 12365: 16,
 12370: 17,
 12372: 18,
 12373: 19,
 12375: 20,
 12377: 21,
 12378: 22,
 12379: 23,
 12380: 24,
 12381: 25,
 12383: 26,
 12384: 27,
 12386: 28,
 12388: 29,
 12390: 30,
 12393: 31,
 12394: 32,
 12395: 33,
 12397: 34,
 12399: 35,
 12401: 36,
 12402: 37,
 12405: 38,
 12406: 39,
 12407: 40,
 12408: 41,
 12409: 42,
 12410: 43,
 12412: 44,
 12413: 45,
 12414: 46,
 12415: 47,
 12417: 48,
 12418: 49,
 12421: 50,
 12422: 51,
 12423: 52,
 12424: 53,
 12425: 54,
 12426: 55,
 12427: 56,
 12428: 57,
 12429: 58,
 12431: 59,
 12432: 60,
 12433: 61,
 12434: 62,
 12435: 63,
 12436: 64,
 12437: 65,
 12438: 66,
 12441: 67,
 12444: 68,
 12447: 69,
 12449: 70,
 12450: 71,
 12451: 72,
 12453: 73,
 12455: 74,
 12456: 75,
 12457: 76,
 12458: 77,
 12461: 78,
 12462: 79,
 12463: 80,
 12464: 81,
 12465: 82,
 12468: 83,
 1

In [55]:
user_to_product = merged_df[['CustomerID','Product Name','Quantity']]
user_to_product = user_to_product.groupby(['CustomerID','Product Name']).agg({'Quantity':'sum'}).reset_index()
user_to_product.tail()

,CustomerID,Product Name,Quantity
138397,18287,Sport-Tek Ladies PosiCharge Competitor Tee,24
138398,18287,Ultra Sleek And Spacious Pearl White Lacquer 1...,6
138399,18287,"Union 3"" Female Ports Stainless Steel Pipe Fit...",12
138400,18287,awesome since 1948 - 69th birthday gift t-shir...,4
138401,18287,billyboards Porcelain Menu Chalkboard,6


In [56]:
product_to_feature = merged_df[['Product Name','Customer Segment','Quantity']]
product_to_feature = product_to_feature.groupby(['Product Name','Customer Segment']).agg({'Quantity':'sum'}).reset_index()
product_to_feature.head()

,Product Name,Customer Segment,Quantity
0,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Corporate,712
1,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Middle class,272
2,"""In Vinyl W.e Trust"" Rasta Quote Men's T-shirt",Small Business,388
3,"""Soccer"" Vinyl Graphic - Large - Ivory",Corporate,1940
4,"""Soccer"" Vinyl Graphic - Large - Ivory",Middle class,1418


# train test split

In [48]:
user_to_product_train,user_to_product_test = train_test_split(user_to_product,test_size=0.33, random_state=42)

In [58]:
print("Training set size:")
print(user_to_product_train.shape)
print("Test set size:")
print(user_to_product_test.shape)

Training set size:
(92729, 3)
Test set size:
(45673, 3)


In [51]:
def interactions(data, row, col, value, row_map, col_map):
    
    #converting the row with its given mappings
    row = data[row].apply(lambda x: row_map[x]).values
    #converting the col with its given mappings
    col = data[col].apply(lambda x: col_map[x]).values
    value = data[value].values
    #returning the interaction matrix
    return coo_matrix((value, (row, col)), shape = (len(row_map), len(col_map)))


In [52]:
# generate user_item_interaction_matrix for train data
user_to_product_interaction_train = interactions(user_to_product_train, "CustomerID", 
                                                    "Product Name", "Quantity", user_to_index_mapping, item_to_index_mapping)

# generate item_to_feature interaction
product_to_feature_interaction = interactions(product_to_feature, "Product Name", "Customer Segment","Quantity",item_to_index_mapping, feature_to_index_mapping)

In [59]:
# generate user_item_interaction_matrix for test data
user_to_product_interaction_test = interactions(user_to_product_test, "CustomerID","Product Name", "Quantity", user_to_index_mapping,item_to_index_mapping)

In [54]:
print(user_to_product_interaction_train)

  (2124, 230)	10
  (1060, 268)	16
  (1633, 295)	3
  (113, 45)	20
  (3100, 44)	54
  (3110, 25)	4
  (2133, 126)	6
  (525, 176)	2
  (2711, 45)	26
  (281, 487)	25
  (1378, 349)	27
  (2356, 18)	12
  (3248, 6)	24
  (1295, 339)	1
  (1229, 4)	120
  (3153, 36)	24
  (2441, 54)	1
  (1522, 124)	4
  (1436, 15)	8
  (3613, 137)	12
  (265, 324)	1
  (3309, 220)	24
  (2943, 3)	18
  (2021, 407)	5
  (2516, 187)	18
  :	:
  (64, 8)	24
  (3406, 109)	1
  (3219, 12)	12
  (2196, 96)	25
  (3409, 236)	12
  (2216, 172)	1
  (2720, 60)	5
  (121, 326)	6
  (1670, 63)	31
  (1555, 4)	6
  (22, 112)	12
  (1670, 8)	15
  (1736, 41)	10
  (1101, 256)	8
  (415, 152)	12
  (3303, 154)	2
  (2955, 8)	16
  (2276, 27)	1
  (3612, 527)	12
  (1446, 299)	192
  (2888, 238)	60
  (3140, 35)	2
  (2727, 591)	7
  (3451, 194)	1
  (3203, 15)	4


# Model building on training set
Parameters:
Loss=warp
epochs=1 
num_threads=4

In [30]:
# initialising model with warp loss function
model_with_features = LightFM(loss = "warp")

# fitting the model with hybrid collaborative filtering + content based (product + features)
start = time.time()
#===================


model_with_features.fit_partial(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=1, 
          num_threads=4,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 0.11 seconds


In [31]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))



time taken = 0.24 seconds
average AUC without adding item-feature interaction = 0.17


# Model building on training set
Parameters:
Loss=logistic
epochs=1 
num_threads=4

In [32]:
# initialising model with warp loss function
model_with_features = LightFM(loss = "logistic")

# fitting the model with hybrid collaborative filtering + content based (product + features)
start = time.time()
#===================


model_with_features.fit_partial(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=1, 
          num_threads=4,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 0.11 seconds


In [33]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.22 seconds
average AUC without adding item-feature interaction = 0.89


# Model building on training set
Parameters:
Loss=bpr
epochs=1 
num_threads=4

In [34]:
# initialising model with warp loss function
model_with_features = LightFM(loss = "bpr")

# fitting the model with hybrid collaborative filtering + content based (product + features)
start = time.time()
#===================


model_with_features.fit_partial(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=1, 
          num_threads=4,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 0.12 seconds


In [35]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.22 seconds
average AUC without adding item-feature interaction = 0.38


# Model building on training set
Parameters:
Loss=logistic
epochs=10 
num_threads=20

In [36]:
model_with_features = LightFM(loss = "logistic")

# fitting the model with hybrid collaborative filtering + content based (product + features)
start = time.time()
#===================


model_with_features.fit_partial(user_to_product_interaction_train,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=10, 
          num_threads=20,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 0.77 seconds


In [37]:
start = time.time()
#===================
auc_with_features = auc_score(model = model_with_features, 
                        test_interactions = user_to_product_interaction_test,
                        train_interactions = user_to_product_interaction_train, 
                        item_features = product_to_feature_interaction,
                        num_threads = 4, check_intersections=False)
#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

print("average AUC without adding item-feature interaction = {0:.{1}f}".format(auc_with_features.mean(), 2))


time taken = 0.25 seconds
average AUC without adding item-feature interaction = 0.89


In [38]:
def train_test_merge(training_data, testing_data):

    # initialising train dict
    train_dict = {}
    for row, col, data in zip(training_data.row, training_data.col, training_data.data):
        train_dict[(row, col)] = data

    # replacing with the test set

    for row, col, data in zip(testing_data.row, testing_data.col, testing_data.data):
        train_dict[(row, col)] = max(data, train_dict.get((row, col), 0))


    # converting to the row
    row_list = []
    col_list = []
    data_list = []
    for row, col in train_dict:
        row_list.append(row)
        col_list.append(col)
        data_list.append(train_dict[(row, col)])

    # converting to np array

    row_list = np.array(row_list)
    col_list = np.array(col_list)
    data_list = np.array(data_list)

    return coo_matrix((data_list, (row_list, col_list)), shape = (training_data.shape[0], training_data.shape[1]))

In [39]:
user_to_product_interaction = train_test_merge(user_to_product_interaction_train, user_to_product_interaction_test)

In [40]:
user_to_product_interaction

<3647x797 sparse matrix of type '<class 'numpy.float64'>'
	with 138402 stored elements in COOrdinate format>

# Final Model after combining the train and test data
Parameters:
Loss=warp
epochs=10 
num_threads=20

In [64]:
# retraining the final model with combined dataset

final_model = LightFM(loss = "warp",no_components=30)

# fitting to combined dataset

start = time.time()
#===================

final_model.fit(user_to_product_interaction,
          user_features=None, 
          item_features=product_to_feature_interaction, 
          sample_weight=None, 
          epochs=10, 
          num_threads=20,
          verbose=False)

#===================
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 3.46 seconds


In [65]:
def get_recommendations(model,user,items,user_to_product_interaction_matrix,user2index_map,product_to_feature_interaction_matrix):

# getting the userindex

    userindex = user2index_map.get(user, None)

    if userindex == None:
        return None

    users = userindex

    # products already bought

    known_positives = items[user_to_product_interaction_matrix.tocsr()[userindex].indices]
    print('User index =',users)

    # scores from model prediction
    scores = model.predict(user_ids = users, item_ids = np.arange(user_to_product_interaction_matrix.shape[1]),item_features=product_to_feature_interaction_matrix)

    # top items

    top_items = items[np.argsort(-scores)]

    # printing out the result
    print("User %s" % user)
    print("     Known positives:")

    for x in known_positives[:10]:
        print("                  %s" % x)


    print("     Recommended:")

    for x in top_items[:10]:
        print("                  %s" % x)


In [66]:
get_recommendations(final_model,17017,items,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 2888
User 17017
     Known positives:
                  Ganma Superheroes Ordinary Life Case For Samsung Galaxy Note 5 Hard Case Cover
                  MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options
                  Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches
                  MightySkins Skin Decal Wrap Compatible with OtterBox Sticker Protective Cover 100's of Color Options
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  MightySkins Skin Decal Wrap Compatible with Lenovo Sticker Protective Cover 100's of Color Options
                  Ebe Reading Glasses Mens Womens Tortoise Bold Rectangular Full Frame Anti Glare grade ckbdp9088
                  Window Tint Film Chevy (back doors) DIY
                  Union 3" Female Ports Stainless Steel Pipe Fitting
                  Ebe Wo

In [67]:
get_recommendations(final_model,18287,items,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 3646
User 18287
     Known positives:
                  MightySkins Skin Decal Wrap Compatible with Nintendo Sticker Protective Cover 100's of Color Options
                  Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lace Silicone Top Band CT Wheat II - Ankle 8-8.75 inches
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  Union 3" Female Ports Stainless Steel Pipe Fitting
                  Ebe Women Reading Glasses Reader Cheaters Anti Reflective Lenses TR90 ry2209
                  MightySkins Skin Decal Wrap Compatible with Apple Sticker Protective Cover 100's of Color Options
                  New Way 1120 - Crewneck Your Job Could Be Worse Toilet Paper Sweatshirt 4XL Light Pink
                  6pc Boy Formal Necktie Black & White Suit Set Satin Bow tie Baby Sm-20 Teen
                  Buckle-Down PC-W30348-NM Cameroon Flags Plastic Clip Collar
                  Business Essentials 8" 

In [68]:
get_recommendations(final_model,13933,items,user_to_product_interaction,user_to_index_mapping,product_to_feature_interaction)

User index = 1000
User 13933
     Known positives:
                  MightySkins Skin Decal Wrap Compatible with DJI Sticker Protective Cover 100's of Color Options
                  Ebe Women Reading Glasses Reader Cheaters Anti Reflective Lenses TR90 ry2209
                  MightySkins Skin Decal Wrap Compatible with Smok Sticker Protective Cover 100's of Color Options
                  3 1/2"W x 20"D x 20"H Funston Craftsman Smooth Bracket, Douglas Fir
                  Ebe Reading Glasses Mens Womens Blue Silver Temples Anti Glare remarkable grade ckb8147
                  Port Authority J335 Hooded Core Soft Shell Jacket, Dress Blue Navy/ Battleship Grey, XS
                  Buckle-Down PC-W30348-NM Cameroon Flags Plastic Clip Collar
                  Business Essentials 8" x 8" x 5" Corrugated Mailers, 12-Pack
                  6 1/4 x 6 1/4 Gatefold Invitation - Mandarin Orange (500 Qty.)
                  Awkward Styles Sugar Skull Shirts for Men Jolly Roger Skull and Crossbo